# Assignment-1: Transformations and representations

Team Name: Robo-Knights

Roll number: 2019111007, 2019112002

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np
import math
import copy

INFO - 2021-09-16 00:15:44,695 - utils - NumExpr defaulting to 8 threads.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [2]:
def display(to_display):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(to_display)
    vis.run()
    vis.destroy_window()

In [3]:
# 1 visualising the stanford bunny file in mesh format
mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")
display(mesh)
# o3d.visualization.draw_geometries([mesh])

# 2 Convert mesh to point cloud 
pcd = mesh.sample_points_uniformly(number_of_points=50000)
# o3d.visualization.draw_geometries([pcd])
display(pcd)

# Setting the color of the point cloud
pcd.paint_uniform_color([0.3,0,0.5])
display(pcd)
# o3d.visualization.draw_geometries([pcd])

In [4]:
# 3 Display the axes while plotting.
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])

# Setting a predefined viewing angle 
o3d.visualization.draw_geometries([pcd, coord_frame],
                                  zoom=1.0,
                                  front=[0, 0, 5],
                                  lookat=[0, 0, 0],
                                  up=[0, 1, 0])

In [5]:
# coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
#     size=0.1, origin=[0, 0, 0])

# 4
# rotate
pcd_r = copy.deepcopy(pcd)
pcd_r.rotate(pcd.get_rotation_matrix_from_xyz((np.pi / 2, 0, 0)),
              center=(0, 0, 0))
o3d.visualization.draw_geometries([pcd_r, coord_frame])

# translate
pcd_tr = copy.deepcopy(pcd_r).translate((1, 0, 0))
o3d.visualization.draw_geometries([pcd_tr, coord_frame])

# scale
pcd_str = copy.deepcopy(pcd_tr)
pcd_str.scale(0.5,center=pcd_str.get_center())
o3d.visualization.draw_geometries([pcd_str, coord_frame])

# transform 
# it shows the final and initial position of the bunny
T = np.eye(4)
T[:3, :3] = pcd.get_rotation_matrix_from_xyz((0, np.pi / 3, np.pi / 2))
T[0, 3] = 1
T[1, 3] = 0.5
pcd_t = copy.deepcopy(pcd).transform(T)
o3d.visualization.draw_geometries([pcd, pcd_t, coord_frame])

In [6]:
# 5
# Save the point cloud as bunny.pcd.
o3d.io.write_point_cloud("str.pcd", pcd_str)
o3d.io.write_point_cloud("transform.pcd", pcd_t)

True

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [7]:
def Rx(theta):
  return np.matrix([[ 1, 0           , 0           ],
                   [ 0, math.cos(theta),-math.sin(theta)],
                   [ 0, math.sin(theta), math.cos(theta)]])
  
def Ry(theta):
  return np.matrix([[ math.cos(theta), 0, math.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-math.sin(theta), 0, math.cos(theta)]])
  
def Rz(theta):
  return np.matrix([[ math.cos(theta), -math.sin(theta), 0 ],
                   [ math.sin(theta), math.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

def getRotationMatrix(alpha, beta, gamma):

    rotation_matrix = np.matmul(Rx(alpha),np.matmul(Ry(beta),Rz(gamma)))
    return rotation_matrix

In [8]:
# matrix1_1 = getRotationMatrix(0, 0, 2*np.pi/9)
# print(matrix1_1)
# matrix1_2 = getRotationMatrix(0, 7*np.pi/18, 0)
# print(matrix1_2)
# matrix1_3 = getRotationMatrix(np.pi/6, 0, 0)
# print(matrix1_3)

# final = np.matmul(matrix1_2, matrix1_3)
# final = np.matmul(matrix1_1, final )
# print(final, "\n")

# matrix2_1 = pcd.get_rotation_matrix_from_xyz((0, 0, 2*np.pi/9))
# print(matrix2_1)
# matrix2_2 = pcd.get_rotation_matrix_from_xyz((0, 7*np.pi/18, 0))
# print(matrix2_2)
# matrix2_3 = pcd.get_rotation_matrix_from_xyz((np.pi/6, 0, 0))
# print(matrix2_3)
# final = np.matmul(matrix2_2, matrix2_3)
# final = np.matmul(matrix2_1, final )
# print(final, "\n")
matrix1 = getRotationMatrix(np.pi/6, 7*np.pi/18, 2*np.pi/9)
matrix2 = pcd.get_rotation_matrix_from_xyz((np.pi/6, 7*np.pi/18, 2*np.pi/9))
# matrix = getRotationMatrix(np.pi/6, 7*np.pi/18, 2*np.pi/9)
# matrix = getRotationMatrix(-np.pi/18, np.pi/2, 0)
# print("Euler rotation matrix for [0,0,30]:")
# print(matrix)
# print(final)
print("Matrix Computed using the function getRotationMatrix:")
print(matrix1)
print("Matrix Computed using inbuilt function:")
print(matrix2)

Matrix Computed using the function getRotationMatrix:
[[ 0.26200263 -0.21984631  0.93969262]
 [ 0.91659355  0.36140256 -0.17101007]
 [-0.30201139  0.90612129  0.29619813]]
Matrix Computed using inbuilt function:
[[ 0.26200263 -0.21984631  0.93969262]
 [ 0.91659355  0.36140256 -0.17101007]
 [-0.30201139  0.90612129  0.29619813]]


In [9]:
# 2
import scipy
from scipy.optimize import fsolve

M = np.array([[0.26200263, -0.19674724, 0.944799], [0.21984631, 0.96542533, 0.14007684], [-0.93969262, 0.17101007, 0.29619813]])
N = np.array([[0., -0.173648178, 0.984807753], [0., 0.984807753, 0.173648178], [-1, 0., 0.]])

# Assuming the vector to be rotated as [1,1,1]
ini_vec = np.array([[1,1,1]])
# Calculating roatated vectors for M and N respectively
final_vec_M = np.matmul(M, ini_vec.T)
final_vec_N = np.matmul(N, ini_vec.T)
# matrix = np.matmul(matrix, ini_vec.T)
# print(Rotated matrices matrix)
# print(final_vec_M)
# print(final_vec_N)


In [10]:
# Definig fsolve function for M
def func1(x):
    return [np.cos(x[0])*np.cos(x[1])*ini_vec[0,0] + (np.cos(x[0])*np.sin(x[1])*np.sin(x[2]) - np.sin(x[0])*np.cos(x[2]))*ini_vec[0,1] + (np.cos(x[0])*np.sin(x[1])*np.cos(x[2]) + np.sin(x[0])*np.sin(x[2]))*ini_vec[0,2] - 1.01005439,
            np.sin(x[0])*np.cos(x[1])*ini_vec[0,0] + (np.sin(x[0])*np.sin(x[1])*np.sin(x[2]) + np.cos(x[0])*np.cos(x[2]))*ini_vec[0,1] + (np.sin(x[0])*np.sin(x[1])*np.cos(x[2]) - np.cos(x[0])*np.sin(x[2]))*ini_vec[0,2] - 1.32534848,
            -np.sin(x[1])*ini_vec[0,0] + np.cos(x[1])*np.sin(x[2])*ini_vec[0,1] + np.cos(x[1])*np.cos(x[2])*ini_vec[0,2] + 0.47248442 ]

In [11]:
# Definig fsolve function for N
def func2(x):
    return [np.cos(x[0])*np.cos(x[1])*ini_vec[0,0] + (np.cos(x[0])*np.sin(x[1])*np.sin(x[2]) - np.sin(x[0])*np.cos(x[2]))*ini_vec[0,1] + (np.cos(x[0])*np.sin(x[1])*np.cos(x[2]) + np.sin(x[0])*np.sin(x[2]))*ini_vec[0,2] - 0.81115958,
            np.sin(x[0])*np.cos(x[1])*ini_vec[0,0] + (np.sin(x[0])*np.sin(x[1])*np.sin(x[2]) + np.cos(x[0])*np.cos(x[2]))*ini_vec[0,1] + (np.sin(x[0])*np.sin(x[1])*np.cos(x[2]) - np.cos(x[0])*np.sin(x[2]))*ini_vec[0,2] - 1.15845593,
            -np.sin(x[1])*ini_vec[0,0] + np.cos(x[1])*np.sin(x[2])*ini_vec[0,1] + np.cos(x[1])*np.cos(x[2])*ini_vec[0,2] + 1.0] 

In [12]:
# Calculating the Euler angles from the ratation matrix
def returnEulerAngles(expr, initialValue):
    root = fsolve(expr, initialValue)
    return root

In [13]:
# Finding the true values of the Euler angles from the rotation matrix
def rotationMatrixToEulerAngles(R) :
    
    if R[2,0]*R[2,0] == 1:
        alpha = 0
        if R[2,0] == -1:
            beta = np.pi/2
            gamma = np.arctan2(R[0,1], R[1,1]) 
        else:
            beta = - np.pi/2
            gamma = - np.arctan2(R[0,1], R[1,1])
    else:
        beta = np.arctan2(-R[2,0], math.sqrt(R[0,0]*R[0,0] + R[1,0]*R[1,0]))
        alpha = np.arctan2(R[1,0]/np.cos(beta), R[0,0]/np.cos(beta))
        gamma = np.arctan2(R[2,1]/np.cos(beta), R[2,2]/np.cos(beta))

    return np.array([alpha, beta, gamma])

# Euler angle ZYX notation used which outputs the actual correct rotation angles in the order Z, Y, X respectively
Correct_root1 = rotationMatrixToEulerAngles(M)
Correct_root2 = rotationMatrixToEulerAngles(N)

print("The real root of M: ", Correct_root1)
print("The real root of N: ", Correct_root2, "\n")
# All values printed in radians
print("Roots found using fsolve method:\n")
# Initializing close to the actual root
root1_1 = returnEulerAngles(func1,[2*np.pi/9.7, 7*np.pi/17.6, np.pi/6.4])
# Initializing randomly
root1_2 = returnEulerAngles(func1,[0, 0, 0])
# Initializing randomly
root1_3 = returnEulerAngles(func1,[np.random.randint(-90, 90)*np.pi/180, np.random.randint(0, 180)*np.pi/180, np.random.randint(0, 180)*np.pi/180])

# Initializing close to the actual root
root2_1 = returnEulerAngles(func2,[0.01, np.pi/2.3, -np.pi/18.1])
# Initializing randomly
root2_2 = returnEulerAngles(func2,[0,0,0])
# Initializing randomly
root2_3 = returnEulerAngles(func2,[np.random.randint(-90, 90)*np.pi/180, np.random.randint(0, 180)*np.pi/180, np.random.randint(0, 180)*np.pi/180])

print("Solving rotation angles for matrix M:")
print("Initializing close to the actual root: ", root1_1)
print("Initializing all the values to zero: ", root1_2)
print("Initializing randomly: ", root1_3)

print("\nSolving rotation angles for matrix N: ")
print("Initializing close to the actual root: ", root2_1)
print("Initializing all the values to zero: ", root2_2)
print("Initializing randomly: ", root2_3)

The real root of M:  [0.6981317  1.22173048 0.52359878]
The real root of N:  [ 0.          1.57079633 -0.17453293] 

Roots found using fsolve method:

Solving rotation angles for matrix M:
Initializing close to the actual root:  [0.70108685 1.2219993  0.52711513]
Initializing all the values to zero:  [44.15651886 26.20554166 50.12496341]
Initializing randomly:  [0.32979438 1.1471521  0.07073858]

Solving rotation angles for matrix N: 
Initializing close to the actual root:  [ 0.10080303  1.57079633 -0.0737299 ]
Initializing all the values to zero:  [40.29491637 39.26990818 40.12038344]
Initializing randomly:  [146.30797444 139.80087309  -4.66300586]


/home/pranjali/anaconda3/envs/mr_assignment1/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


We observe that if the values are initialized close to the correct values, fsolve solves it correctly than when initialized far away from the true values. 

3. Euler angles have a major deficiency, and that is, that it is possible, in some rotation sequences, to reach a situation where two of the three Euler angles cause rotation around the same axis of the object. In the case for example, rotation around the x axis becomes indistinguishable in its effect from rotation around the z axis, so the z and x axis angles collapse into one transformation, and the rotation reduces from three degrees of freedom to two. So the loss of one degree of freedom in 3-D is called Gimbal lock. It makes Euler angles unsuited for practical applications. 

In [14]:
# 4
def animate_rotation(alpha, beta, gamma, pcd, vis):
    
    one_degree = np.pi/180
    steps = 100
    
    forward_step = np.array([one_degree*alpha/steps, one_degree*beta/steps, one_degree*gamma/steps])
    for i in range(steps-1):
        
        pcd.rotate(np.transpose(pcd.get_rotation_matrix_from_xyz(((i)*forward_step[0],(i)*forward_step[1],(i)*forward_step[2]))),
              center=pcd.get_center())
        pcd.rotate(pcd.get_rotation_matrix_from_xyz((i+1)*forward_step),
              center=pcd.get_center())
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        
pcd__ = copy.deepcopy(pcd)

vis = o3d.visualization.Visualizer()
vis.create_window()
# pcd__.rotate(pcd__.get_rotation_matrix_from_xyz((np.pi/6,np.pi/2,0)))
vis.add_geometry(pcd__)
pcd_copy = copy.deepcopy(pcd__)
pcd_2 = copy.deepcopy(pcd)
pcd_2.paint_uniform_color([0.9,0,0.1])
# pcd_2.rotate(pcd_2.get_rotation_matrix_from_xyz((0,np.pi/2,np.pi/6)))
vis.add_geometry(pcd_2)


animate_rotation(0,90,30,pcd__, vis)
animate_rotation(30,90,0,pcd_2, vis)
vis.run()
vis.destroy_window()

## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

1. Euler angles faces challenges of Gimbal lock. But quaternions can easily overcome that as the computation just involves the multiplication of two numbers (with 4 elements each) which is also faster than matrix multiplication (3,3 = 9 elements for one matrix). For example in aircraft and rocket, changes in its orientation can be given by three rotations known as pitch, roll and yaw, represented by three arrays of numbers called matrices. But when two rotation axes align, an ambiguity arises that can result in catastrophic loss of control (gimbal lock). In such situation using quaternions is helpful. The quaternion algebra also allows us to easily compose rotations. Also they provide convenient mathematical notation for representing spatial orientations and rotations of elements in three dimensional space. Specifically, they encode information about an axis-angle rotation about an arbitrary axis. 

In [15]:
# 2 
# Get quaternions from the rotation matrix
def rotMat_to_quaternion(rotMat):
#     print(rotMat[0,0])
#     print(rotMat[1,1])
#     print(rotMat[2,2])
    trace_rotMat = rotMat[0,0] + rotMat[1,1] + rotMat[2,2]
#     print(trace_rotMat)


    q0 = math.sqrt((trace_rotMat + 1)/4)
    q0_inv = 0.25/q0
    q1 = (rotMat[2,1] - rotMat[1,2])*q0_inv
    q2 = (rotMat[0,2] - rotMat[2,0])*q0_inv
    q3 = (rotMat[1,0] - rotMat[0,1])*q0_inv
    
    return np.array([q0, q1, q2, q3])

In [16]:
# 2 
# Get rotation matrix from quaternions
def quaternion_to_rotMat(q):
    rotMatrix = np.zeros((3, 3))
    
    rotMatrix[0,0] = q[0]*q[0] + q[1]*q[1] - 0.5
    rotMatrix[0,1] = q[1]*q[2] - q[0]*q[3] 
    rotMatrix[0,2] = q[0]*q[2] + q[1]*q[3]
    rotMatrix[1,0] = q[0]*q[3] + q[1]*q[2]
    rotMatrix[1,1] = q[0]*q[0] + q[2]*q[2] - 0.5
    rotMatrix[1,2] = q[2]*q[3] - q[0]*q[1]
    rotMatrix[2,0] = q[1]*q[3] - q[0]*q[2]
    rotMatrix[2,1] = q[0]*q[1] + q[2]*q[3]
    rotMatrix[2,2] = q[0]*q[0] + q[3]*q[3] - 0.5
    
    rotMatrix = 2*rotMatrix
    return rotMatrix

In [17]:
# Getting the rotation matrices from the rotation angles
rotmax1 = getRotationMatrix(0, 0, np.pi/6)
rotmax2 = getRotationMatrix(np.pi/6, 0, 0)

print("Rotaion matrix 1 for [0, 0, 30]: \n", rotmax1, "\n")
print("Rotaion matrix 2 for [30, 0, 0]: \n", rotmax2 ,"\n")

# get quaternions for the above rotation matrix
q1 = rotMat_to_quaternion(rotmax1)
q2 = rotMat_to_quaternion(rotmax2)    
print("Quaternion corrersponding to rotation matrix 1: ", q1)
print("Quaternion corrersponding to rotation matrix 2: ", q2, "\n")

# check if the R matirx calculated both ways is same
# Converting from quaternion to rotaion matrix
rotmax1_q1 = quaternion_to_rotMat(q1)
rotmax2_q2 = quaternion_to_rotMat(q2)
print("Observe the rotation matrices calculated from the rotation angles and quaternions is same.\n")
print("Rotation matrix corrersponding to quaternion 1: \n", rotmax1_q1)
print("Rotation matrix corrersponding to quaternion 2: \n", rotmax2_q2)

# check of the quaternion calculated through R matrix in same


Rotaion matrix 1 for [0, 0, 30]: 
 [[ 0.8660254 -0.5        0.       ]
 [ 0.5        0.8660254  0.       ]
 [ 0.         0.         1.       ]] 

Rotaion matrix 2 for [30, 0, 0]: 
 [[ 1.         0.         0.       ]
 [ 0.         0.8660254 -0.5      ]
 [ 0.         0.5        0.8660254]] 

Quaternion corrersponding to rotation matrix 1:  [0.96592583 0.         0.         0.25881905]
Quaternion corrersponding to rotation matrix 2:  [0.96592583 0.25881905 0.         0.        ] 

Observe the rotation matrices calculated from the rotation angles and quaternions is same.

Rotation matrix corrersponding to quaternion 1: 
 [[ 0.8660254 -0.5        0.       ]
 [ 0.5        0.8660254  0.       ]
 [ 0.         0.         1.       ]]
Rotation matrix corrersponding to quaternion 2: 
 [[ 1.         0.         0.       ]
 [ 0.         0.8660254 -0.5      ]
 [ 0.         0.5        0.8660254]]


In [18]:
# Multiply two quaternions
def Multiply_2_quaternions(q, p):
    
    rotq00 = q[0]*p[0] - q[1]*p[1] - q[2]*p[2] - q[3]*p[3] 
    rotq01 = q[0]*p[1] + q[1]*p[0] + q[2]*p[3] - q[3]*p[2]  
    rotq02 = q[0]*p[2] + q[2]*p[0] - q[1]*p[3] + q[3]*p[1] 
    rotq03 = q[0]*p[3] + q[3]*p[0] + q[1]*p[2] - q[2]*p[1] 
    
    return np.array([rotq00, rotq01, rotq02, rotq03])

In [25]:
# 3 Perform matrix multiplication of two 3×3 rotation matrices and perform the same transformation in the quaternion space.
# Performing multiplication in quaternion space for the above two quaternions
resq = Multiply_2_quaternions(q2, q1)
print("Resultant vector after quaternion multiplication:", resq ,"\n")

# Multiplying above two rotation matrices
rotq = np.matmul(rotmax2, rotmax1)
print("Product of two rotation matrices: \n", rotq ,"\n")

# Finding the equivalent quaternion for the above product 
rotq_q = rotMat_to_quaternion(rotq)
print("Finding the quaternion from the result of the matrix multiplication which is equal to the result of the multiplication  of the two quaternions performed in quaternion space:", rotq_q)
# Since rotq_q = resq, hence same transformation is performed in quaternion space

Resultant vector after quaternion multiplication: [ 0.9330127  0.25      -0.0669873  0.25     ] 

Product of two rotation matrices: 
 [[ 0.8660254 -0.5        0.       ]
 [ 0.4330127  0.75      -0.5      ]
 [ 0.25       0.4330127  0.8660254]] 

Finding the quaternion from the result of the matrix multiplication which is equal to the result of the multiplication  of the two quaternions performed in quaternion space: [ 0.9330127  0.25      -0.0669873  0.25     ]


In [26]:
# 4
def norm(q):
    return (math.sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2))
def slerp(p0, p1, t):
    omega = 0
    if norm(p0) == 0:
        omega = np.arccos(np.dot(p0, p1/norm(p1)))
    elif norm(p1) == 0:
        omega = np.arccos(np.dot(p0/norm(p0), p1))
    else:  
        omega = np.arccos(np.dot(p0/norm(p0), p1/norm(p1)))
    so = np.sin(omega)
    return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")

# Convert mesh to point cloud 
pcd = mesh.sample_points_uniformly(number_of_points=50000)

q0 = rotMat_to_quaternion(pcd.get_rotation_matrix_from_xyz((0,0,0)))
q1 = rotMat_to_quaternion(pcd.get_rotation_matrix_from_xyz((0,np.pi/2,0)))
q_inter = slerp(q0,q1,0.1)

#initial bunny
pcd_initial = copy.deepcopy(pcd)
pcd_initial.paint_uniform_color([0.1,0,0.8])
pcd_initial.translate((-0.3,0,0))

#interpolated 
pcd_inter = copy.deepcopy(pcd)
pcd_inter.paint_uniform_color([0.8,0,0.1])

pcd_final = copy.deepcopy(pcd)
pcd_final.paint_uniform_color([0.5,0,0.5])
pcd_final.translate((0.3,0,0))

pcd_initial.rotate(quaternion_to_rotMat(q0),center=pcd_initial.get_center())
pcd_inter.rotate(quaternion_to_rotMat(q_inter),center=pcd_inter.get_center())
pcd_final.rotate(quaternion_to_rotMat(q1),center=pcd_final.get_center())

o3d.visualization.draw_geometries([pcd_initial, pcd_inter, pcd_final])

In the above visualization, the blue image is the initial bunny, red is the intermediate bunny and violet the final bunny. Using spherical linear interpolation (slerp) formula, we've calculated the interpolated quaternion and used it to display the intermediate bunny.

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

1. The exponential map transforms skew-symmetric matrices into orthogonal matrices, and every rotation matrix can be represented as the matrix exponential of some skew-symmetric matrix. Also rotation is parameterized by a rotation in a three-dimensional Euclidean space by just two quantities which are a unit vector indicating the direction of an axis of rotation, and an angle θ describing the magnitude of the rotation about the axis. Only two numbers (not three or more which are used in other representations), are needed to define the direction of a unit vector at the origin because the magnitude of that vector is constrained. It represents that any rotation or sequence of rotations of a rigid body in a three-dimensional space is equivalent to a pure rotation about a single fixed axis.

In [27]:
# 2
# Obtaining the rotation matrix form the given vector and theta
def matEx(w, t):
    mod_w = math.sqrt(w[0]*w[0] + w[1]*w[1] + w[2]*w[2]) 
    x = w[0]/mod_w
    y = w[1]/mod_w
    z = w[2]/mod_w
    ct = np.cos(t)
    st = np.sin(t)
    
    rotMatrix = np.zeros((3, 3))
    
    rotMatrix[0,0] = x*x*(1-ct) + ct
    rotMatrix[0,1] = x*y*(1-ct) - z*st
    rotMatrix[0,2] = x*z*(1-ct) + y*st
    rotMatrix[1,0] = x*y*(1-ct) + z*st
    rotMatrix[1,1] = y*y*(1-ct) + ct
    rotMatrix[1,2] = y*z*(1-ct) - x*st
    rotMatrix[2,0] = x*z*(1-ct) - y*st
    rotMatrix[2,1] = y*z*(1-ct) + x*st
    rotMatrix[2,2] = z*z*(1-ct) + ct
    
    return rotMatrix
    

In [28]:
# Perform matrix exponentiation
w = np.array([2,1,15])
theta = 4.1364
R = matEx(w, theta)
print("Rotation matrix for rotation along the direction of the vector [2,1,15] with an angle 237 degree:\n", R)

Rotation matrix for rotation along the direction of the vector [2,1,15] with an angle 237 degree:
 [[-0.51780074  0.84292002  0.14617876]
 [-0.81605629 -0.53794854  0.21133741]
 [ 0.25677718 -0.00985943  0.96642034]]


In [29]:
# 3
# Obtaining the logarithmic map
def logMap(R):
    tr = R[0,0] + R[1,1] + R[2,2]
    
#     print(tr)
    if (R == np.eye(3)).all():
        theta = 0
        w = np.array(["nan", "nan", "nan"])
    elif tr ==  -1:
#         print("In")
        theta = np.pi
        r1 = 1/math.sqrt(2*(1 + R[2][2]))
        r2 = 1/math.sqrt(2*(1 + R[1][1]))
        r3 = 1/math.sqrt(2*(1 + R[0][0]))
        w1 = r1*R[0][2] + r2*R[0][1] + r3*(1 + R[0][0])
        w2 = r1*R[1][2] + r2*(1 + R[1][1]) + r3*R[1][0]
        w3 = r1*(1 + R[2][2]) + r2*R[2][1] + r3*R[2][0]
        w = np.array(w1, w2, w3)
    else:
#         print("INNN")
        theta = np.arccos(0.5*(tr-1))
        w_square = (1/np.sin(theta))*(R - R.T)
        w = np.array([w_square[2,1], w_square[0,2], w_square[1,0]])
    return theta , w;
    

In [30]:
given_R = np.array([[0.1, -0.9487, 0.3], [0.9487, 0, -0.3162], [0.3, 0.3162, 0.9]])
# print(given_R)
theta_out, R_out  = logMap(given_R)
# Angle values given in radians
print("Rotation vector for the given rotation matrix: ", R_out)
print("Rotation angle for the given rotation matrix: ", theta_out)

Rotation vector for the given rotation matrix:  [0.6324 0.     1.8974]
Rotation angle for the given rotation matrix:  1.5707963267948966


# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 


1.  For the memory compression in octomap, all nodes are converted to their maximum likelihood probabilities, followed by tree pruning. This yields an even compression and less memory requirements. Each node in the tree stores in addition to the data payload the coordinate of its center location, its voxel size, and pointers to its children. But this can use substantial amount of memory. Since the node location and its voxel size can be reconstructed while traversing the octree, we do not explicitly store this information in the nodes to reduce the memory overhead. Any leaf node in the octree only stores the mapping data itself and one (null) pointer. Inner nodes additionally store eight pointers to their children. This saves 2/3rd of memory compared to allocating 8 pointers for each node. Octree maps can be recursively encoded as a compact bit stream while saving it as a map file. Each node is represented only by the eight labels of its children. Beginning at the root node, each child that is not a leaf is recursively added to the bit stream. Leaf nodes do not have to be added since they can be reconstructed from their label during the decoding process. The last row only contains leafs so no further nodes are added. This way octomap is memory efficient. 


2. Whenever the log-odds value of a voxel reaches either the lower bound or the upper bound, we consider the node as stable in our approach. Which means stable nodes have been measured free or occupied with high confidence.  If all children of an inner tree node are stable leaf nodes with the same occupancy state, then the children can be pruned. When measurement to be integrated contradict the state of the corresponding inner node, then its children are regenerated and the map is updated accordingly. In octomaps we use one child pointer per node that points to an array of eight pointers. This array is only allocated if the node indeed has children and is not allocated for leaf nodes.


3. We would likely use an octomap instead of point cloud when:
    1. We have limited memory. Point clouds need to store large amounts of measurement points and therefore require a lot of meanory and are not memory efficient. 
    2. To distinguish between obstacle-free and unmapped areas as point clouds do not allow to differentiate between obstacle-free and unmapped areas and provide no means of fusing multiple measurements probabilistically.
    3. Also, point clouds are only suitable for high precision sensors in static environments and when unknown areas do not need to be represented. So where we need to represent arbitrary environment octomaps is a better choice. 

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?



1. Signed Distance Function (SDF) gives us a distance of point X from the boundary of a surface. The major advantage of this function is to determine if a point lies inside the boundary of the surface or outside the boundary. The surface is at those locations where SDF function is 0. The function has positive values at points x inside the surface, it decreases in value as approaches the boundary of the surface where the signed distance function is zero, and it takes negative values outside of the surface.

2. For SDF we store the distance and the weight values for each voxel and these values are updated in each iteration. When the ray is projected from the sensor origin to the surface, we compute projective distance from each voxel. When we have multiple sensor measurements  we update the voxel map with the distances to the nearest surface. We fuse the information from every camera and we perform weighted average to update each cell. The weight is initialized to zero, and is incremented every time we get new sensor measurements. The weight value acts as an measure of confidence for the distance calculated. The weight and distance updation strategy is different for different implementations. The voxels that are being intersected with the ray are updated in every iteration. Finally differnt aggregation strategies are used to update the map information like weighted average to produce the final map.       

3. Occupancy grid maps or voxels store the occupancy probability for each voxels whereas SDF will get actual distance value to the surface and indicates if the voxel is present inside or outside the surface. In SDF the noise cancels out over multiple measurements and zero crossing can be extracted at sub-voxel accuracy (least square estimate). In applications such a path planning where knowing distance to a surface is of high importance SDF is better implemented than voxels. In occupancy grid only single value of occupancy probability is stored and ESDF cannot be directly computed data can only be compressed moderately whereas in TSDF distance and weight values are stored and ESDF can be calculated directly and also allows higher compression of the data.     

4. SDF representations proved to be especially efﬁcient forfont rendering than point clouds. Also in point clouds outside and inside of a surface is not defined. Point clouds do not describe topology and are not suitable for producing watertight surfaces. Also point cloud is represented explicitly where geometry is not stored explicitly but rather a function defined over a surface is embedded whereas in point clouds the geometry is stored explicitly as a list of points. Using point clouds closed and multivalued surfaces are difficult to represent. Also occupancy information is not provided by point clouds. So, SDF is advantageous when modeling shapes with complex and/or changing topology and provides very suitable representation for constructive solid geometry.      

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/